### Import all required modules

In [2]:
import gym
import numpy as np
import random
import matplotlib.pyplot as plt
from tqdm import tqdm
from itertools import product

#### create a environment for problem defination

In [6]:
# create emvirnoment
env = gym.make("FrozenLake-v1")

# render function used to visualize state of problem.
env.render()


SFFF
FHFH
FFFH
HFFG


#### do some basic operations with env and get info about env

In [8]:
# apply 2nd action on current environment
env.reset()
env.step(2)

(0, 0.0, False, {'prob': 0.3333333333333333})

In [9]:
# action space is total possible actions
print("Action space : ",env.action_space)

# observation space is total possible states
print("state space : ",env.observation_space)

Action space :  Discrete(4)
state space :  Discrete(16)


In [10]:
total_states = env.observation_space.n
total_actions = env.action_space.n

In [11]:
# it shows different information for each pair of states and actions.
# information like probability , reward, next state and is it goal state or not?
env.P

{0: {0: [(0.3333333333333333, 0, 0.0, False),
   (0.3333333333333333, 0, 0.0, False),
   (0.3333333333333333, 4, 0.0, False)],
  1: [(0.3333333333333333, 0, 0.0, False),
   (0.3333333333333333, 4, 0.0, False),
   (0.3333333333333333, 1, 0.0, False)],
  2: [(0.3333333333333333, 4, 0.0, False),
   (0.3333333333333333, 1, 0.0, False),
   (0.3333333333333333, 0, 0.0, False)],
  3: [(0.3333333333333333, 1, 0.0, False),
   (0.3333333333333333, 0, 0.0, False),
   (0.3333333333333333, 0, 0.0, False)]},
 1: {0: [(0.3333333333333333, 1, 0.0, False),
   (0.3333333333333333, 0, 0.0, False),
   (0.3333333333333333, 5, 0.0, True)],
  1: [(0.3333333333333333, 0, 0.0, False),
   (0.3333333333333333, 5, 0.0, True),
   (0.3333333333333333, 2, 0.0, False)],
  2: [(0.3333333333333333, 5, 0.0, True),
   (0.3333333333333333, 2, 0.0, False),
   (0.3333333333333333, 1, 0.0, False)],
  3: [(0.3333333333333333, 2, 0.0, False),
   (0.3333333333333333, 1, 0.0, False),
   (0.3333333333333333, 0, 0.0, False)]},
 2:

### Training and evaluation of Agent

- first we train agent on set of exmaples -> from the training we got the q-table which helps us to select appropriate action.

- Then we evaluate that q-table on the set of examples as the part of evaluation.

In [12]:

def q_learning(alpha,gamma,epsilon,episodes = 5000):
    """ This function try to achieve optimize q-table
        values by learning set of examples
    """
    
    # initialize emptry q-table
    q_table = np.zeros((total_states,total_actions))
    all_trip_lengths = []
    
    for i in range(episodes):
        # reset env
        state = env.reset()
        # initialize all zeros when env reset
        reward = 0
        penalty = 0
        # trip length is measure that shows how many steps made by agent to reach to the goal.
        trip_length = 0
        goal = False
        while not goal:
            # add some randomness in training to make it more efficient
            if random.uniform(0,1) < epsilon:
                action = env.action_space.sample()
            else:
                # select action with max benefit.
                action = np.argmax(q_table[state])
            # apply most optimal action.
            next_state, reward, goal, info = env.step(action)
            old_q_val = q_table[state,action]
            next_max_val = np.max(q_table[next_state])
            # compute new q value from old q value and reward.
            new_q_val = (1-alpha)*old_q_val + alpha * (reward + gamma * next_max_val)
            q_table[state,action]= new_q_val
            trip_length += 1
        
#         if(i%1000 == 0):
#             print("Episode : ",i)
#             print("Trip length : ",trip_length)
        all_trip_lengths.append(trip_length)
    return q_table


def evaluate_q_learning(q_table,episodes=30):
    """ This function is used to evaluate the q-table
        Q-table which we got after training for some number of
        episodes that table will be evaluated.
    """
    # for evaluation purpose we have 1 measure.
    # trip -> number of steps made by agent to reach to the goal - it should be minimum
    
    # procedure of evaluation is almost same as training but only difference is
    # here our q-table is already trained so no need to add randomness
    # to select action we only use q-table not an epsilon value.
    total_trip_length = 0
    
    for i in range(episodes):
        # initialize variables when env reset
        state = env.reset()
        reward = 0
        trip_length = 0
        goal = False
        
        # set max trip length is 30 when trip length exceed max value it should be breaked.
        # otherwise it goes into the infinite loop.
        while not goal and trip_length < 30:
            
            # select optimal action
            action = np.argmax(q_table[state])
            next_state, reward, goal, info = env.step(action)
            old_q_val = q_table[state,action]
            next_max_val = np.max(q_table[next_state])
            new_q_val = (1-alpha)*old_q_val + alpha * (reward + gamma * next_max_val)
            q_table[state,action]= new_q_val
            trip_length += 1
            
        total_trip_length += trip_length
    
    results = total_trip_length/episodes
    return results

#### Hyper Parameter tuning

##### Three parameters can be tuned here:
- alpha or learning rate
- gamma or discount function
- epsilon (factor of randomness)

In [13]:
alpha_list = [0.2,0.3,0.4,0.5]
gamma_list = [0.3,0.4,0.5,0.6]
epsilon = [0.2,0.3,0.4]
results = {}
for alpha,gamma,epsilon in tqdm(list(product(alpha_list, gamma_list, epsilon)),ncols=100):
    q_table = q_learning(alpha,gamma,epsilon,episodes=5000)
    results[f"a={alpha},g={gamma},e={epsilon}"] = evaluate_q_learning(q_table)

100%|███████████████████████████████████████████████████████████████| 48/48 [00:56<00:00,  1.14s/it]


In [14]:
# get top three pairs of hyper parameter which gives best solution with lowest trip length.
sorted(results.items(),key = lambda x: x[1])[:3]

[('a=0.2,g=0.5,e=0.4', 5.0),
 ('a=0.4,g=0.5,e=0.3', 5.033333333333333),
 ('a=0.3,g=0.6,e=0.4', 5.4)]

In [15]:
# sometimes different results will be found for same parameters values just because of randomness.
# for ex: alpha=0.2,gamma=0.5,epsilon=0.4, i got average trip length is 5.0
# when i again train agent with same alpha, gamma and epsilon value then i will get different average trip length
# that makes more difficult the process of selecting best parameters value.